In [41]:
import numpy as np
import pandas as pd
import torchvision.io as io
import matplotlib.pyplot as plt
import torchaudio
import torch
import torch.nn as nn

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
print(f"Sample Rate: {bundle.sample_rate}")
print(f"Labels: {bundle.get_labels()}")
model = bundle.get_model().to(device)
print(model.__class__)

Using device cuda
Sample Rate: 16000
Labels: ('-', '|', 'E', 'T', 'A', 'O', 'N', 'I', 'H', 'S', 'R', 'D', 'L', 'U', 'M', 'W', 'C', 'F', 'G', 'Y', 'P', 'B', 'V', 'K', "'", 'X', 'J', 'Q', 'Z')
<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [27]:
waveform, sample_rate = torchaudio.load("temp/data/M003_angry_level_1_001.wav")
vid = io.read_video(f"temp/data/M003_front_angry_level_1_001.mp4", pts_unit="sec",output_format="TCHW")[0]
target =  torch.from_numpy(np.load(f"temp/data/M003_front_angry_level_1_001.npy"))
waveform = waveform.to(device)
if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

transformer_model = torch.nn.Transformer(nhead=16, num_encoder_layers=12)

/home/sowwn/SSD/mah_space/2024/W2T/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Model

In [23]:
with torch.inference_mode():
    emission, _ = model(waveform)

In [38]:
print(f"Target shape: {target.shape}")
print(f"Video shape: {vid.shape}")
print(f"Input shape: {waveform.shape}")
print(f"Vec shape: {emission.shape}")



Target shape: torch.Size([98, 478, 2])
Video shape: torch.Size([98, 3, 256, 256])
Input shape: torch.Size([2, 51542])
Vec shape: torch.Size([2, 160, 29])


In [36]:
torch.unsqueeze(emission,0).shape

torch.Size([1, 2, 160, 29])

In [44]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [45]:
emission.shaep

torch.Size([2, 160, 29])